# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to read
data = "../WeatherPy\output"

# Read Data File and store into Pandas DataFrames
weather_data = pd.read_csv(data)
weather_data


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,50.49,58.0,85.0,8.68,NZ,1.669870e+09
1,1,albany,42.6001,-73.9662,36.18,73.0,57.0,5.01,US,1.669870e+09
2,2,vaini,-21.2000,-175.2000,82.56,83.0,75.0,9.22,TO,1.669870e+09
3,3,jamestown,42.0970,-79.2353,27.72,97.0,100.0,19.57,US,1.669869e+09
4,4,hasaki,35.7333,140.8333,48.29,76.0,100.0,20.22,JP,1.669870e+09
...,...,...,...,...,...,...,...,...,...,...
73,73,kununurra,-15.7667,128.7333,100.38,16.0,97.0,10.36,AU,1.669870e+09
74,74,guanare,9.0418,-69.7421,77.02,73.0,11.0,8.05,VE,1.669870e+09
75,75,mahebourg,-20.4081,57.7000,81.25,69.0,40.0,6.91,MU,1.669870e+09
76,76,ribeira grande,38.5167,-28.7000,60.78,80.0,5.0,5.99,PT,1.669870e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
len(weather_data)
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  78 non-null     int64  
 1   City        78 non-null     object 
 2   Lat         71 non-null     float64
 3   Lng         71 non-null     float64
 4   Max Temp    71 non-null     float64
 5   Humidity    71 non-null     float64
 6   Cloudiness  71 non-null     float64
 7   Wind Speed  71 non-null     float64
 8   Country     70 non-null     object 
 9   Date        71 non-null     float64
dtypes: float64(7), int64(1), object(2)
memory usage: 6.2+ KB


In [4]:
new = weather_data.dropna()

In [5]:
len(new)
new.head(70)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,50.49,58.0,85.0,8.68,NZ,1.669870e+09
1,1,albany,42.6001,-73.9662,36.18,73.0,57.0,5.01,US,1.669870e+09
2,2,vaini,-21.2000,-175.2000,82.56,83.0,75.0,9.22,TO,1.669870e+09
3,3,jamestown,42.0970,-79.2353,27.72,97.0,100.0,19.57,US,1.669869e+09
4,4,hasaki,35.7333,140.8333,48.29,76.0,100.0,20.22,JP,1.669870e+09
...,...,...,...,...,...,...,...,...,...,...
73,73,kununurra,-15.7667,128.7333,100.38,16.0,97.0,10.36,AU,1.669870e+09
74,74,guanare,9.0418,-69.7421,77.02,73.0,11.0,8.05,VE,1.669870e+09
75,75,mahebourg,-20.4081,57.7000,81.25,69.0,40.0,6.91,MU,1.669870e+09
76,76,ribeira grande,38.5167,-28.7000,60.78,80.0,5.0,5.99,PT,1.669870e+09


In [6]:
# Configure gmaps with unique API key
gmaps.configure(api_key=g_key)

In [7]:
# Use the Lat and Lng as locations and Humidity as the weight.

subset = new[['Lat', 'Lng']]
coordinates = [tuple(x) for x in subset.to_numpy()]
coordinates


[(-46.1927, 168.8643),
 (42.6001, -73.9662),
 (-21.2, -175.2),
 (42.097, -79.2353),
 (35.7333, 140.8333),
 (59.7035, 30.7861),
 (-9.8, -139.0333),
 (-38.0023, -57.5575),
 (36.8197, 21.7049),
 (65.282, -126.8329),
 (37.1041, -113.5841),
 (22.0752, -159.319),
 (-28.7667, 114.6),
 (64.3833, -173.3),
 (-42.7826, 147.0587),
 (-33.918, 25.5701),
 (-46.1333, 167.6833),
 (-8.9266, 125.4092),
 (-33.0153, 27.9116),
 (-23.1203, -134.9692),
 (-42.8794, 147.3294),
 (-19.0595, -169.9187),
 (51.3223, 42.7678),
 (64.5011, -165.4064),
 (77.484, -69.3632),
 (10.3997, -75.5144),
 (46.7208, -92.1041),
 (-33.9258, 18.4232),
 (55.7435, -97.8558),
 (64.571, 30.5767),
 (64.2539, -15.2082),
 (-5.1989, -35.4608),
 (-23.7, 133.8833),
 (-54.8, -68.3),
 (27.0377, 14.4283),
 (58.8053, -122.7002),
 (-0.1934, -74.7819),
 (69.4541, -133.0374),
 (-34.4187, 19.2345),
 (71.9667, 102.5),
 (47.1053, -104.7125),
 (41.3712, -73.414),
 (20.5169, -13.0499),
 (32.6667, -17.1),
 (-32.5, 137.7667),
 (3.4836, 34.1492),
 (19.7297, 

In [8]:
# Customize the size of the figure
fig_1_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_1 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig_1.add_layer(markers)
fig_1


Figure(layout=FigureLayout(height='420px'))

In [9]:
humidity = new["Humidity"].astype(float)

In [10]:
# Customize the size of the figure
fig_2_layout = {
    'width': '500px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
# Plot Heatmap
fig_2 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)


# Add layer
fig_2.add_layer(heat_layer)

# Display figure
fig_2

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# A max temperature higher than 80 degrees but lower than 100
# Wind speed less than 10 m/s
# Zero cloudiness

preferences = new.loc[(new['Cloudiness']==0) & (new['Max Temp']>=80) & (new['Max Temp']<=100) & (new['Wind Speed']<=10) ]
preferences

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,alice springs,-23.7,133.8833,85.55,14.0,0.0,8.05,AU,1.669870e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

hotel_df = new[['City', 'Country', 'Lat', 'Lng', 'Humidity']]


In [13]:

hotel = []
hotel_df

for index, row in hotel_df.iterrows():
    location = str(row["Lat"]) + "," + str(row["Lng"])
    params = {
    "location": location,
    "keyword": "hotel",
    "radius": "5000",
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # Start try except incase a hotel cannot be located nearby
    # extract results
    
    try:
        response = requests.get(base_url, params=params).json()
        results = response['results']
        #hotel.append(response["results"][0]["name"])
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except:
        hotel.append("None Nearby")

 

C:\Users\AdminLocal\AppData\Local\Temp\ipykernel_13760\3798325744.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


In [14]:
#add "Hotel Name" column to hotel
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mataura,NZ,-46.1927,168.8643,58.0,Southland Organic Farmstay
1,albany,US,42.6001,-73.9662,73.0,NaN
2,vaini,TO,-21.2000,-175.2000,83.0,Toni's Guest House
3,jamestown,US,42.0970,-79.2353,97.0,DoubleTree by Hilton Jamestown
4,hasaki,JP,35.7333,140.8333,76.0,Inubosaki Hotel
...,...,...,...,...,...,...
73,kununurra,AU,-15.7667,128.7333,16.0,The Kimberley Grande Resort
74,guanare,VE,9.0418,-69.7421,73.0,Nuevo Hotel Coromoto
75,mahebourg,MU,-20.4081,57.7000,69.0,Holiday Inn Mauritius Mon Tresor
76,ribeira grande,PT,38.5167,-28.7000,80.0,Quinta da Meia Eira


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
#fig_2.add_layer(markers)
fig_2.add_layer(hotel_layer)
fig_2



Figure(layout=FigureLayout(height='420px'))